# Steam Game Data Analysis
**Lily Wei and Fiona McSherry**

## Question
Video games are a significant part of today's society. They act as a form of entertainment, a way for people to interact socially across the globe, a tool for education, and they can even be used to create works of art. 

Video games have become a diverse and influential form of entertainment, with a large range of different user experiences for people to explore to their own preferences. Because of this huge diversity, it is very difficult to pinpoint what exactly makes a popular and loved game.
In this essay, we explore the question: **what makes a popular video game?**

## Method
To answer this question we used data from the game platform Steam. Steam is a video game digital distribution service and storefront that allows users to purchase games and play them on any compatible device logged into their account. 

From Steam we gather data from the list of the top 100 most played games based upon the number of current players ( https://store.steampowered.com/charts/mostplayed ). From this list we pulled data about each individual game on the list, including its top 3 genres, price, peak number of players, number of reviews, and positivity of reviews. This helps us find the correlation between known popular games and answer our question.

### Data Retrieval

To obtain this data we utilized the `requests` python library, the `bs4` (Beautiful Soup) library, the `pandas` library, the `Playwright` library, and the `re` library. 
We find all of the following in this section with the function in the file `get_data.py`.

* We retrieved the html of the most played list by using the `Playwright` library to launch the url of the most played list in a browser. We then gather that pages content and cast it as a Beautiful soup object using `bs4` to parse the gathered html into a useable format. 

* Then, we find the important body text from this html, since for the purpose of our research we only care for the actual list, not the other extra information on the webpage. We did this by finding all instances of "`tbody`" in the html.

* Next we gather the url to each game in the list. We did this by using the `re` library to find all instances of the url structure in the body text.

*Now to retrieving our actual data points and categories.* 

Our data categories are: **name, genres, price, peak players, number of reviews, and positivity of reviews.**

* To get the name of each game, we used each game link and found the last occuring index of a slash, the rest of the link is the game's title.

* To find the price of each game, we can return to the body text of the most played list and search through each row of the table by finding the html element `tr`. There are two cases of prices: *Free to Play* or a *$* followed by a number value. Finding this allows us to add each price to a list.

* To find the peak number of players we look to the the rest of the table following the price, so we take from the index of the price and create a partition cutting out the part of the table before the price. Then we search what remains for the html element `td`, which specifies table data cells, with the value contained inside.  

# `Lily write here` how you got the peak pls fix mine lol

* To find the number of reviews *and* the positivity of them, we examine a Beautiful Soup object of the individual game's html content. Among this, there is a class called `nonresponsive_hidden responsive_reviewdesc` which contains a string of the number of reviews and the percentage of their positivity. The percent of positive reviews always directly follows the character `"-"` and ends with the character `"%"`, so the value between those two characters is the positivity. This percent is always follows by `"of the"` and the number of reviews. So we can retrieve the number of reviews by taking the value between the end of the `"of the"` and before string `"user reviews"`.

* To get the top 3 genres of each game, we examine the Beautiful Soup object of that game page's html content. The class `glance_tags popular_tags` contains the many genres associated with that game, so we find the occurance of this. However this can be quite a lot so we chose to only look at the 3 most relevant in our analysis. 

### Data Analysis

Now to analyze this data in order to answer our research question. The following are done using functions found in `analyze_data.py` and `main.py`.